In [1]:
from modules.option_trading import OptionPosition
from modules.vol_mdn import MDNVol, Data

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (15, 10)

In [2]:
data = Data()

In [3]:
x, y = data.get_training_data()
oos_x = data.get_forecast_data()

In [4]:
model = MDNVol(x, y)
dist = model.output_dist(oos_x)

Epoch 0 complete. Loss: 26114.40625
Epoch 10 complete. Loss: 12528.17578125
Epoch 20 complete. Loss: 9715.4814453125
Epoch 30 complete. Loss: 8128.7197265625
Epoch 40 complete. Loss: 7224.7919921875
Epoch 50 complete. Loss: 6631.3037109375
Epoch 60 complete. Loss: 6269.587890625
Epoch 70 complete. Loss: 6180.6015625
Epoch 80 complete. Loss: 5995.51513671875
Epoch 90 complete. Loss: 5842.701171875
Epoch 100 complete. Loss: 5932.482421875
Epoch 110 complete. Loss: 5369.158203125
Epoch 120 complete. Loss: 5091.29638671875
Epoch 130 complete. Loss: 4890.8623046875
Epoch 140 complete. Loss: 4725.12548828125
Epoch 150 complete. Loss: 4602.671875
Epoch 160 complete. Loss: 4485.81640625
Epoch 170 complete. Loss: 4471.2158203125
Epoch 180 complete. Loss: 4436.23193359375
Epoch 190 complete. Loss: 4424.09033203125
Epoch 200 complete. Loss: 4337.56298828125
Epoch 210 complete. Loss: 4328.91455078125
Epoch 220 complete. Loss: 4364.443359375
Epoch 230 complete. Loss: 4210.87548828125
Epoch 240 comp

In [5]:
vols = dist.sample([100000]).squeeze().numpy()
vols = np.sqrt(np.exp(vols))

In [6]:
chain = pd.read_csv("option_chain.csv")
dte = 4
risk_free_rate = .0006

position = OptionPosition(chain, vols, dte, risk_free_rate)

In [7]:
position.calc_kelly()

array([0.66435299])

In [8]:
position.short_put

strike            4010.0
right                  P
bid                  2.3
ask                 2.45
impvol          0.176947
delta          -0.060185
modelprice      2.029832
gamma           0.001537
vega            0.524454
theta          -1.122461
underprice       4128.99
model_value     1.115593
mid_price          2.375
skew_premium    1.259407
Name: 648, dtype: object

In [9]:
position.short_call

strike            4215.0
right                  C
bid                  1.6
ask                 1.75
impvol          0.124385
delta           0.058957
modelprice      1.375982
gamma           0.002151
vega            0.516097
theta          -0.777411
underprice       4129.05
model_value     3.696619
mid_price          1.675
skew_premium   -2.021619
Name: 335, dtype: object